<a href="https://colab.research.google.com/github/Mohammed-Abdul-Rafe-Sajid/MNIST-DL-with-Data-Augmenatation/blob/main/MNIST_with_data_augmenatation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# Datasets with new transform
train_dataset_augmented = MNIST(root='./data', train=True, transform=train_transform_augmented, download=True)
test_dataset = MNIST(root='./data', train=False, transform=test_transform)

# DataLoaders
train_loader_augmented = DataLoader(train_dataset_augmented, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:

import torch.nn as nn
import torch.optim as optim
aug_model = SimpleCC1()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aug_model.to(device)
print(device)
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(aug_model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5
train_accuracy = []
train_loss = []

for epoch in range(num_epochs):
    aug_model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader_augmented:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = aug_model(images)
        loss = criterion(outputs, labels)

        # Backward + optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy calculation
        _, predicted = torch.max(outputs, 1)      # Get predicted class
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader_augmented)
    epoch_acc = 100 * correct / total

    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCC2(nn.Module):  #for RGB input
    def __init__(self):
        super(SimpleCC2, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # now it is compatible with 3 input channels for RGB
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 3 * 3, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 16, 14, 14]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 32, 7, 7]
        x = self.pool(F.relu(self.conv3(x)))  # [B, 64, 3, 3]

        x = x.view(-1, 64 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
from torchvision import transforms

# Augmented training transform
train_transform_augmented = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# keeping it compatible to 3 channels
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # RGB normalization
])

In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# Datasets with new transform
train_dataset_augmented = MNIST(root='./data', train=True, transform=train_transform_augmented, download=True)
test_dataset = MNIST(root='./data', train=False, transform=test_transform)

# DataLoaders
train_loader_augmented = DataLoader(train_dataset_augmented, batch_size=64, shuffle=True)
test_loader_augmented = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
from torchvision.datasets import MNIST

test_dataset = MNIST(root='./data', train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:

import torch.nn as nn
import torch.optim as optim
aug_model = SimpleCC2()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aug_model.to(device)
print(device)
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(aug_model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5
train_accuracy = []
train_loss = []

for epoch in range(num_epochs):
    aug_model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader_augmented:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = aug_model(images)
        loss = criterion(outputs, labels)

        # Backward + optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy calculation
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader_augmented)
    epoch_acc = 100 * correct / total

    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


In [ ]:
def test_model(model, test_loader):
    model.eval()  # Use the model argument, not hardcoded aug_model
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Use the passed-in model
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


test_model(aug_model, test_loader_augmented)


In [ ]:
import matplotlib.pyplot as plt

# Plot training accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracy, marker='o', color='green')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid(True)

# Plot training loss
plt.subplot(1, 2, 2)
plt.plot(train_loss, marker='o', color='red')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Set model to evaluation
aug_model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = aug_model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Generate report
report = classification_report(all_labels, all_preds, digits=10)
print(report)